<a href="https://colab.research.google.com/github/geee05/Fine-Tuning-hugging-face-models/blob/main/roberta_autosequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install --upgrade accelerate

In [ ]:
import torch
import numpy as np
import pandas as pd

!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
import os

from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, AutoModelForSequenceClassification, Trainer

torch.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 133.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

'2.0.1+cu118'

In [ ]:
dataset = pd.read_csv('/content/DAIC_train_3sp_sampled.csv')
tdataset= pd.read_csv('/content/DAIC_test_3sp.csv')

In [ ]:
dataset['Group'].value_counts()

0    5132
1    5132
Name: Group, dtype: int64

In [ ]:
dataset.rename(columns = {'Group':'label'}, inplace = True)

In [ ]:
tdataset.rename(columns = {'Group':'label'}, inplace = True)

In [ ]:
train = Dataset.from_pandas(dataset, preserve_index=False)
test = Dataset.from_pandas(tdataset, preserve_index=False)

In [ ]:
print(train)
print(test)

Dataset({
    features: ['Unnamed: 0', 'path', 'text', 'label'],
    num_rows: 10264
})
Dataset({
    features: ['ID', 'path', 'text', 'label'],
    num_rows: 1914
})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train.map(tokenize_function, batched=True)
tokenized_test_dataset = test.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/10264 [00:00<?, ? examples/s]

Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_train_dataset)
print(tokenized_test_dataset)

Dataset({
    features: ['Unnamed: 0', 'path', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10264
})
Dataset({
    features: ['ID', 'path', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1914
})


In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [13]:
training_args = TrainingArguments("test-trainer",
    evaluation_strategy="epoch",
    eval_steps = 100,
    learning_rate=2e-5
    )   # default arguments for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)  # overwriting MLM roberta-base for sequence binary classification

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

trainer = Trainer(   # specifying trainer class
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()  # starts fine-tuning

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initia

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.698500,0.699025,0.295193,0.455829


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.698500,0.699025,0.295193,0.455829
2,0.688800,0.697894,0.476489,0.422811
3,0.628400,0.730159,0.570010,0.399708


TrainOutput(global_step=3849, training_loss=0.6727312563239336, metrics={'train_runtime': 3245.0437, 'train_samples_per_second': 9.489, 'train_steps_per_second': 1.186, 'total_flos': 8101715616645120.0, 'train_loss': 0.6727312563239336, 'epoch': 3.0})